In [ ]:
import ast
from glob import glob
import re
import numpy as np
import matplotlib.pyplot as plt
from exod.utils.path import data_results
import pandas as pd

from tqdm import tqdm
from exod.utils.path import read_observation_ids
from exod.post_processing.collate_results import read_all_csv_1d, read_all_csv_regions

from astropy.visualization import hist
from astropy.table import Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.units as u

pd.set_option('display.max_rows', 20)

In [ ]:
filepath = '../data/results_combined/t_50s_2_12/'
regex = re.compile(r'\d{10}')

In [ ]:
df_simlist = pd.read_csv('../data/results_combined/t_50s_2_12/EXOD_simlist_2024-04-19_16-08-13.csv')
df_simlist

In [ ]:
df_simlist['status'].value_counts()

In [ ]:
df_evt = read_all_csv_1d(glob_pattern=f'{filepath}*/*/*evt_info.csv')
df_evt['dt'] = pd.to_datetime(df_evt['date'])
df_evt.sort_values('dt', ascending=False)
df_dl  = read_all_csv_1d(glob_pattern=f'{filepath}*/*/*dl_info.csv')
df_dc  = read_all_csv_1d(f'{filepath}*/*/*data_cube_info.csv')
df_regions = read_all_csv_regions(f'{filepath}*/*/*regions.csv')

In [ ]:
def filter_filename(filename, str='PN'):
    try:
        inst = [f for f in filename if str in f][0]
        return inst
    except IndexError:
        return None

In [ ]:
dfs = []
for obsid in tqdm(df_regions['obsid'].unique()):
    subsets = list(glob(f'{filepath}{obsid}/*'))
    for s in subsets:
        csv_evt = glob(f'{s}/*evt_info.csv')[0]
        csv_reg = glob(f'{s}/*regions.csv')[0]
        df_evt = pd.read_csv(csv_evt, index_col=0)
        df_reg = pd.read_csv(csv_reg)

        filename = ast.literal_eval(df_evt.T['filename'].values[0])
        #print(filename)
        df_reg['PN_EVT']   = filter_filename(filename, str='PN')
        df_reg['M1_EVT']   = filter_filename(filename, str='M1')
        df_reg['M2_EVT']   = filter_filename(filename, str='M2')
        df_reg['exposure'] = float(df_evt.T['exposure'].values[0])
        #print(df_evt)
        
        df_reg['obsid'] = obsid
        try:
            df_reg_sub = df_reg[['obsid', 'PN_EVT','M1_EVT','M2_EVT','exposure', 'ra', 'dec', 'ra_deg', 'dec_deg', 'area_bbox', 'intensity_mean']]
            dfs.append(df_reg_sub)
        except:
            print(s)


df_final = pd.concat(dfs, axis=0)
df_final

In [ ]:
df_final['ct_rate'] = df_final['intensity_mean'] / df_final['exposure']

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(15,5))
ax[0].hist(df_final['intensity_mean'], bins=np.linspace(1,2000,100), color='blue')
ax[1].hist(df_final['exposure'], bins=np.linspace(1,150000,100), color='blue')
ax[2].hist(df_final['ct_rate'], np.linspace(0,0.01,100), color='blue')
ax[0].set_xlabel('Region Mean Counts')
ax[1].set_xlabel('Observation Exposure')
ax[2].set_xlabel('Region Mean Count Rates')
plt.show()

In [ ]:
df_regions2 = df_regions[['obsid', 'ra', 'dec', 'intensity_mean']]
df_regions2.sort_values('intensity_mean', ascending=False).reset_index(drop=True)

In [ ]:
df_regions2['obsid'].value_counts()

In [ ]:
# All Regions Plot
sky_coords = SkyCoord(ra=df_regions['ra_deg'].values, dec=df_regions['dec_deg'].values, unit='deg', frame='icrs')
sky_coords

ra_rad = sky_coords.ra.wrap_at(180 * u.deg).radian
dec_rad = sky_coords.dec.radian

# Plotting with Aitoff projection
plt.figure(figsize=(15, 10))
plt.subplot(111, projection='aitoff')

plt.title(f'Detected Regions by EXOD: {len(sky_coords)}')

galactic_coords = sky_coords.galactic
galactic_center = SkyCoord(0, 0, unit="deg", frame="galactic")

# Scatter plot the Galactic coordinates
I_log = np.log10(df_regions['intensity_mean'])
m = plt.scatter(galactic_coords.l.wrap_at(180 * u.deg).radian,
            galactic_coords.b.radian,
            marker='s', label='Detected Regions', s=10.0, c=I_log, cmap='cool')

#plt.scatter(0, 0, marker='+', color='Blue', label='Galactic Center')
plt.colorbar(m, shrink=1.0, orientation='horizontal', label='Log Average Total Counts')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Crossmatching with XMM catalogue For column descriptions see:
# http://xmmssc.irap.omp.eu/Catalogue/4XMM-DR13/4XMM-DR13_Catalogue_User_Guide.html#Catalogue
# Basic energy bands:
# 1 = 0.2 - 0.5 keV
# 2 = 0.5 - 1.0 keV
# 3 = 1.0 - 2.0 keV
# 4 = 2.0 - 4.5 keV
# 5 = 4.5 - 12.0 keV
# Broad energy bands: 	
# 6 = 0.2 - 2.0  keV | soft band, no images made
# 7 = 2.0 - 12.0 keV | hard band, no images made
# 8 = 0.2 - 12.0 keV | total band
# 9 = 0.5 - 4.5  keV | XID band

tab_xmm = Table.read('../data/util/4XMM_slim_DR13cat_v1.0.fits')
tab_xmm

In [ ]:
skycoord_xmm = SkyCoord(ra=tab_xmm['SC_RA'], dec=tab_xmm['SC_DEC'], unit=u.deg)
skycoord_xmm

In [ ]:
# Find nearest matches
coord1 = sky_coords
coord2 = skycoord_xmm

idx, sep2d, d3d = match_coordinates_sky(coord1, coord2)

# Calculate the differences in RA and Dec for the nearest matches
ra_diff  = coord1.ra  - coord2[idx].ra
dec_diff = coord1.dec - coord2[idx].dec

# Get subset for below specifed distance
dist_arcsec  = 20
mask_sub     = sep2d.arcsec < dist_arcsec
ra_diff_sub  = ra_diff[mask_sub]
dec_diff_sub = dec_diff[mask_sub]

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].scatter(ra_diff.arcsec, dec_diff.arcsec,  s=1, color='black')
ax[0].set_title(f'Offset, all sources ({len(ra_diff)})')
ax[0].set_xlabel('Ra offset (arcsec)')
ax[0].set_ylabel('Dec offset  (arcsec)')


ax[1].set_title(f'Offset, sources < {dist_arcsec}" ({len(ra_diff_sub)})')
ax[1].scatter(ra_diff_sub.arcsec, dec_diff_sub.arcsec,  s=1, color='black')
ax[1].set_xlabel('Ra offset (arcsec)')
ax[1].set_ylabel('Dec offset  (arcsec)')
ax[1].set_xlim(-30,30)
ax[1].set_ylim(-30,30)


fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].hist(ra_diff_sub.arcsec, bins=20, color='black')
ax[0].set_xlabel('Ra offset (arcsec)')
ax[1].hist(dec_diff_sub.arcsec, bins=20, color='black', orientation='horizontal')
ax[1].set_xlabel('Dec offset (arcsec)')


In [ ]:
sep_arcsec = sep2d.to(u.arcsec)
h = plt.hist(sep_arcsec, color='black', bins=np.arange(0,200,1)*u.arcsec)
heights, bin_edges = h[0], h[1]
plt.axvline(14.5, color='red', ls='dotted', lw=1.0)
plt.xlabel('Seperation (arcsec)')
plt.ylabel('Count')
plt.xlim(0)
plt.show()


In [ ]:
cmatch = sky_coords.match_to_catalog_sky(skycoord_xmm)
tab_cmatch = Table(cmatch)
tab_cmatch.rename_columns(names=tab_cmatch.colnames, new_names=['idx', 'sep2d', 'dist3d'])
tab_cmatch['sep2d_arcsec'] = tab_cmatch['sep2d'].to(u.arcsec)
tab_cmatch['idx_orig'] = np.arange(len(tab_cmatch))
tab_cmatch

In [ ]:
tab_cmatch_good = tab_cmatch[tab_cmatch['sep2d_arcsec'] <= 20]
tab_cmatch_none = tab_cmatch[tab_cmatch['sep2d_arcsec'] > 20]
tab_xmm_good = tab_xmm[tab_cmatch_good['idx']]
tab_xmm_none = tab_xmm[tab_cmatch_none['idx']]

In [ ]:
# Crossmatch Plot
sc_good = SkyCoord(ra=tab_xmm_good['SC_RA'], dec=tab_xmm_good['SC_DEC'], unit='deg', frame='icrs')
sc_none = sky_coords[tab_cmatch_none['idx_orig']]

def sc_to_gal(sc):
    ra_rad = sc.ra.wrap_at(180 * u.deg).radian
    dec_rad = sc.dec.radian
    return sc.galactic
    
sc_gal_good = sc_to_gal(sc_good)
sc_gal_none = sc_to_gal(sc_none)

# Plotting with Aitoff projection
plt.figure(figsize=(15, 10))
plt.subplot(111, projection='aitoff')
plt.title(f'Detected Regions by EXOD: {len(sky_coords)}')
plt.scatter(sc_gal_good.l.wrap_at(180 * u.deg).radian, sc_gal_good.b.radian, marker='.', label=f'Counterpart Sources {len(tab_xmm_good)}', s=5.0, color='blue')
plt.scatter(sc_gal_none.l.wrap_at(180 * u.deg).radian, sc_gal_none.b.radian, marker='.', label=f'No Counterparts {len(tab_xmm_none)}', s=5.0, color='red')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Plot sources with large seperations
sep_min = 100
mask_sub = abs(dec_diff.arcsec) > sep_min

# All Regions Plot
sky_coords = coord1[mask_sub]
ra_rad = sky_coords.ra.wrap_at(180 * u.deg).radian
dec_rad = sky_coords.dec.radian

# Plotting with Aitoff projection
plt.figure(figsize=(15, 10))
plt.subplot(111, projection='aitoff')
plt.title(f'Sources with abs(Dec) > {sep_min} arcsec')
galactic_coords = sky_coords.galactic
galactic_center = SkyCoord(0, 0, unit="deg", frame="galactic")

# Scatter plot the Galactic coordinates

m = plt.scatter(galactic_coords.l.wrap_at(180 * u.deg).radian,
            galactic_coords.b.radian, marker='s', label='Detected Regions', s=10.0, color='red')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# To find the optimal seperation, plot the histogram of seperations and find the lowest point.

In [ ]:
N_src  = len(tab_cmatch)
N_good = len(tab_cmatch_good)

hist(tab_cmatch_good['sep2d_arcsec'], bins='knuth', histtype='stepfilled', ec='k', fc='#AAAAAA')
plt.title(rf'Distribution of {N_good}/{N_src} regions with with XMM counterparts $\leq$ 20"')
plt.xlabel('Seperation (arcseconds)')

mu  = np.mean(tab_cmatch_good['sep2d_arcsec'])
std = np.std(tab_cmatch_good['sep2d_arcsec'])
plt.axvline(mu, color='red', label=fr'mean={mu:.2f}$\pm${std:.2f}')
plt.axvline(mu+std, color='green')
plt.axvline(mu-std, color='green')
plt.legend()
plt.show()

In [ ]:
N_src  = len(tab_cmatch)
N_good = len(tab_cmatch_none)

hist(tab_cmatch_none['sep2d_arcsec'], bins='knuth', histtype='stepfilled', ec='k', fc='#AAAAAA')
plt.title(rf'Distribution of {N_good}/{N_src} regions with with XMM counterparts > 20"')
plt.xlabel('Seperation (arcseconds)')
plt.yscale('log')
plt.show()

In [ ]:
# Count Rate vs Chi^2 Prob
plt.figure(figsize=(6,6))
plt.scatter(tab_xmm_good['SC_CHI2PROB'], tab_xmm_good['SC_EP_1_FLUX'], marker='.', s=10, color='black', label='SC_EP_1_FLUX')
plt.scatter(tab_xmm_good['SC_CHI2PROB'], tab_xmm_good['SC_EP_5_FLUX'], marker='.', s=10, color='blue', label='SC_EP_5_FLUX')
plt.xlim(xmin=0)
plt.yscale('log')
plt.xlabel('SC_CHI2PROB')
plt.ylabel('FLUX')
plt.legend()
plt.show()

In [ ]:
# Count Rate vs F_var
plt.figure(figsize=(6,6))
plt.scatter(tab_xmm_good['SC_HR1'], tab_xmm_good['SC_EP_1_FLUX'], marker='.', s=1, color='black', label='SC_EP_1_FLUX')
plt.scatter(tab_xmm_good['SC_HR1'], tab_xmm_good['SC_EP_5_FLUX'], marker='.', s=1, color='blue', label='SC_EP_5_FLUX')
plt.xlim(xmin=0)
plt.yscale('log')
#plt.xscale('log')
plt.xlabel('N_DETECTIONS')
plt.ylabel('FLUX')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sub = tab_xmm_good[(tab_xmm_good['SC_CHI2PROB'] > 0.8) & (tab_xmm_good['SC_HR1'] > 0.75)]
plt.scatter(tab_xmm_good['SC_CHI2PROB'], tab_xmm_good['SC_HR1'], marker='.', s=1, color='black')
plt.scatter(sub['SC_CHI2PROB'], sub['SC_HR1'], marker='.', s=20, color='red')
plt.xlim(xmin=0)
#plt.yscale('log')
plt.xlabel('SC_CHI2PROB')
plt.ylabel('SC_HR1')
plt.show()

In [ ]:
plt.hist(df_dc['bti_frac'].astype(np.float32), bins=np.linspace(0,1,100))
plt.show()

In [ ]:
df_evt